In [ ]:

import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

# Configuración
BASE_URL = "https://www.fib.upc.edu/"
OUTPUT_DIR = "scraped_fib_v2"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}
VISITED_URLS = set()

# Crear carpeta base si no existe
os.makedirs(OUTPUT_DIR, exist_ok=True)

def get_page_content(url):
    """Descarga y parsea el contenido HTML de la página."""
    try:
        response = requests.get(url, headers=HEADERS, verify=False, timeout=10)
        
        # Si la web devuelve error, mostrarlo y no continuar
        if response.status_code != 200:
            print(f"⚠️ Error {response.status_code} al acceder a: {url}")
            return None
        
        return BeautifulSoup(response.text, "html.parser")
    
    except requests.exceptions.SSLError:
        print(f"❌ Error SSL en {url}")
        return None
    except requests.exceptions.ConnectionError:
        print(f"❌ Error de conexión en {url}")
        return None
    except requests.exceptions.Timeout:
        print(f"❌ Tiempo de espera agotado en {url}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"❌ Error desconocido en {url}: {e}")
        return None

def sanitize_filename(url):
    """Convierte una URL en un nombre de archivo válido."""
    parsed_url = urlparse(url)
    path = parsed_url.path.strip("/").replace("/", "_") or "index"
    return f"{path}.md"

def extract_text(soup, url):
    """Extrae el contenido de la página en formato Markdown."""
    md_content = f"[⬅ Volver a Inicio]({BASE_URL})\n\n"

    # Título de la página
    title = soup.find("title").text if soup.find("title") else "Sin título"
    md_content += f"# {title}\n\n"

    # Extraer párrafos
    for p in soup.find_all("p"):
        md_content += f"{p.get_text(strip=True)}\n\n"

    # Extraer imágenes con su texto alternativo
    for img in soup.find_all("img"):
        alt_text = img.get("alt", "Imagen sin descripción")
        md_content += f"![{alt_text}]()\n\n"

    # Extraer tablas en formato Markdown
    for table in soup.find_all("table"):
        rows = table.find_all("tr")
        for i, row in enumerate(rows):
            cols = [col.get_text(strip=True) for col in row.find_all(["td", "th"])]
            md_content += " | ".join(cols) + "\n"
            if i == 0:
                md_content += " | ".join(["---"] * len(cols)) + "\n"

    # Extraer enlaces internos
    md_content += "\n## Enlaces internos\n"
    for link in soup.find_all("a", href=True):
        href = urljoin(url, link["href"])
        parsed_href = urlparse(href)

        # Solo seguir enlaces internos (mismo dominio)
        if parsed_href.netloc == urlparse(BASE_URL).netloc and href not in VISITED_URLS:
            filename = sanitize_filename(href)
            md_content += f"- [{link.get_text(strip=True)}]({filename})\n"
    
    return md_content

def save_markdown(content, filename):
    """Guarda el contenido en un archivo Markdown."""
    file_path = os.path.join(OUTPUT_DIR, filename)
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(content)
    print(f"✅ Guardado: {file_path}")

def scrape_page(url):
    """Scrapea una página y sigue sus enlaces internos."""
    if url in VISITED_URLS:
        return
    VISITED_URLS.add(url)

    soup = get_page_content(url)
    if not soup:
        return
    
    filename = sanitize_filename(url)
    markdown_content = extract_text(soup, url)
    save_markdown(markdown_content, filename)

    # Recorrer enlaces internos
    for link in soup.find_all("a", href=True):
        href = urljoin(url, link["href"])
        parsed_href = urlparse(href)

        # Seguir solo enlaces internos
        if parsed_href.netloc == urlparse(BASE_URL).netloc and href not in VISITED_URLS:
            scrape_page(href)

def main():
    print(f"🚀 Scrapeando {BASE_URL} y sus subpáginas...\n")
    scrape_page(BASE_URL)
    print("\n🎉 ¡Scraping completado!")

if __name__ == "__main__":
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)  # Ocultar warnings de SSL
    main()


🚀 Scrapeando https://www.fib.upc.edu/ y sus subpáginas...

✅ Guardado: scraped_fib_v2\index.md
✅ Guardado: scraped_fib_v2\index.md
✅ Guardado: scraped_fib_v2\ca_noticies_rss.rss.md


C:\Users\KennyAlejandroPoma\AppData\Local\Programs\Python\Python311\Lib\html\parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


✅ Guardado: scraped_fib_v2\ca.md
✅ Guardado: scraped_fib_v2\ca.md
✅ Guardado: scraped_fib_v2\en.md
✅ Guardado: scraped_fib_v2\en.md
✅ Guardado: scraped_fib_v2\en_noticies_rss.rss.md
✅ Guardado: scraped_fib_v2\es.md
✅ Guardado: scraped_fib_v2\es.md
✅ Guardado: scraped_fib_v2\es_noticies_rss.rss.md
✅ Guardado: scraped_fib_v2\es_contact.md
✅ Guardado: scraped_fib_v2\es_contact.md
✅ Guardado: scraped_fib_v2\ca_contact.md
✅ Guardado: scraped_fib_v2\ca_contact.md
✅ Guardado: scraped_fib_v2\en_contact.md
✅ Guardado: scraped_fib_v2\en_contact.md
✅ Guardado: scraped_fib_v2\en_studies.md
✅ Guardado: scraped_fib_v2\en_studies.md
✅ Guardado: scraped_fib_v2\ca_estudis.md
✅ Guardado: scraped_fib_v2\ca_estudis.md
✅ Guardado: scraped_fib_v2\es_estudios.md
✅ Guardado: scraped_fib_v2\es_estudios.md
✅ Guardado: scraped_fib_v2\es_estudios_grados.md
✅ Guardado: scraped_fib_v2\es_estudios_grados.md
✅ Guardado: scraped_fib_v2\ca_estudis_graus.md
✅ Guardado: scraped_fib_v2\ca_estudis_graus.md
✅ Guardado: scra